In [ ]:
import os
import cv2
import time
from PIL import Image
import numpy as np
import tensorflow as tf
import tensorflow_hub as hub
import matplotlib.pyplot as plt
#ONLY WORK FOR ONLINE COLAB
from google.colab import drive
drive.mount('/content/drive')

os.environ["TFHUB_DOWNLOAD_PROGRESS"] = "True"

SAVED_MODEL_PATH = "https://tfhub.dev/captain-pool/esrgan-tf2/1"

def capture_frames(video_path, frame_path, frame_format):
    # Open the video file
    video = cv2.VideoCapture(video_path)

    # Read the video
    fps = video.get(cv2.CAP_PROP_FPS)
    width = int(video.get(cv2.CAP_PROP_FRAME_WIDTH))
    height = int(video.get(cv2.CAP_PROP_FRAME_HEIGHT))

    frame_count = 0
    while True:
        # Read a frame from the video
        ret, frame = video.read()

        if not ret:
            break

        # Save the frame as an image
        frame_filename = f"{frame_count}.{frame_format}"  # Specify the frame filename
        frame_filepath = f"{frame_path}/{frame_filename}"  # Specify the frame file path
        cv2.imwrite(frame_filepath, frame)

        frame_count += 1

    video.release()
    cv2.destroyAllWindows()


def create_video_from_frames(frame_path, output_path, fps, offset):
    frame_filenames = os.listdir(frame_path)
    first_frame = cv2.imread(os.path.join(frame_path, frame_filenames[0]))
    height, width, _ = first_frame.shape
    # Create a VideoWriter object to save the frames as a video
    fourcc = cv2.VideoWriter_fourcc(*"mp4v")  # Specify the codec (e.g., XVID, MJPG, mp4v)
    output_video = cv2.VideoWriter(output_path, fourcc, fps, (width, height))

    # Loop through each frame file and write it to the output video
    for i in range(offset,offset+len(frame_filenames)):
        filename = str(i)+'.png'
        frame_filepath = os.path.join(frame_path,filename)
        # frame_filepath = os.path.join(frame_path, frame_filename)
        frame = cv2.imread(frame_filepath)
        output_video.write(frame)

    # Release the resources
    output_video.release()
    cv2.destroyAllWindows()

def preprocess_image(image_path):
  """ Loads image from path and preprocesses to make it model ready
      Args:
        image_path: Path to the image file
  """
  hr_image = tf.image.decode_image(tf.io.read_file(image_path))
  # If PNG, remove the alpha channel. The model only supports
  # images with 3 color channels.
  if hr_image.shape[-1] == 4:
    hr_image = hr_image[...,:-1]
  hr_size = (tf.convert_to_tensor(hr_image.shape[:-1]))
  hr_image = tf.image.crop_to_bounding_box(hr_image, 0, 0, hr_size[0], hr_size[1])
  hr_image = tf.cast(hr_image, tf.float32)
  return tf.expand_dims(hr_image, 0)

def save_image(image, filename):
  #  image: 3D image tensor. [height, width, channels]
  if not isinstance(image, Image.Image):
    image = tf.clip_by_value(image, 0, 255)
    image = Image.fromarray(tf.cast(image, tf.uint8).numpy())
  image.save("%s" % filename)
  print("Saved as %s" % filename)


model = hub.load(SAVED_MODEL_PATH)

In [ ]:

video_path    = 'path/to/video'
output_path   = 'path/to/output'

frame_path    = 'path/to/frame'
sr_frame_path = 'path/to/sr_frame'
lr_frame_path = 'path/to/lr_frame'


#frames per second
fps = 30

#make directory
if not os.path.exists(frame_path):
  os.makedirs(frame_path)

if not os.path.exists(sr_frame_path):
  os.makedirs(sr_frame_path)

if not os.path.exists(lr_frame_path):
  os.makedirs(lr_frame_path)

#when call the function you can set the format of frames captured
capture_frames(video_path,frame_path,'png')


In [ ]:
#downsampling the videos by 4 for evaluating the programme
images = os.listdir(frame_path)
# Loop over all the images
for i in range(0,len(images)):
    frame_name = str(i)+'.png'
    img = Image.open(os.path.join(frame_path,frame_name))
    width,height = img.size
    img_lr = img.resize((width//4,height//4))
    img_lr.save(os.path.join(lr_frame_path,frame_name))


In [ ]:
start = time.time()

lr_frame = os.listdir(lr_frame_path)

for i in range(0, len(lr_frame)):
    frame_name = str(i)+'.png'
    tmp_path = os.path.join(lr_frame_path,frame_name)
    tmp_image = preprocess_image(tmp_path)
    sr_image = model(tmp_image)
    sr_image = tf.squeeze(sr_image)
    sr_save_path = os.path.join(sr_frame_path,frame_name)
    print('frame '+ str(i) + 'done')
    save_image(sr_image,sr_save_path)


print("Time Taken: %f" % (time.time() - start))


In [ ]:
output_video_path   = output_path + 'filename'

fps = 30
# offset of this function is the number index of the frame started with.
create_video_from_frames(sr_frame_path,output_video_path,fps,0)


In [ ]:
from skimage.metrics import structural_similarity

start = time.time()
PSNR = []
SSIM = []
EXE_TIME = []
INDEX = []

for i in range(0,len(lr_frame)):
  tmp_filename = str(i)+'.png'
  im1 = cv2.imread('path to original frames'+ tmp_filename)
  im2 = cv2.imread('path to SR frames'+ tmp_filename)
  im1 = im1.astype('float32')
  im2 = im2.astype('float32')
  if i % 10 == 0:
    print(i)

  # Compute SSIM between twoimages
  ssim = structural_similarity(im1,im2, multichannel=True, gaussian_weights=True, sigma=1.5 , use_sample_covariance=False, data_range=255)
  psnr = cv2.PSNR(im1,im2)
  ssim1 = tf.image.ssim(im1, im2, max_val=255, filter_size=11,
                     filter_sigma=1.5, k1=0.01, k2=0.03)
  psnr1 = tf.image.psnr(im1, im2, max_val=255)

  PSNR.append(psnr)
  SSIM.append(ssim)

print("Time Taken: %f" % (time.time() - start))
psnr_mean = np.mean(PSNR)
ssim_mean = np.mean(SSIM)
print(ssim_mean)
print(psnr_mean)

# for i in range(len(INDEX)):
#   print('Image 0'+ str(INDEX[i])+': ExecutionTime:' + str(format(EXE_TIME[i],'.4f')) + ' PSNR:' + str(format(PSNR[i],'.4f')) + ' SSIM: ' + str(format(SSIM[i],'.4f')))